In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-t9hnolcl
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-t9hnolcl
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 781ff5b76ba6c4c2d80dcbbec9983e147613cc71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.1.0-py3-none-any.whl size=8011 sha256=e2b2acbb79d726227ace2ffdeb18b901e4debb9cf9419b13a35a389a655ee4ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-0sgs275b/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [3]:
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmpgyazkicd".


**Question1 (Q1)**

In [ ]:

%%writefile Q1_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>
using namespace std;

// Kernel definition
__global__ void ArrAdd(float *arr, int size,float *result)
{
    // declares a shared memory segment that is accessible by all threads in the same block. More on this later.
extern __shared__ float partialSum[];

unsigned int thread_id = blockIdx.x * blockDim.x + threadIdx.x ;

unsigned int t = threadIdx.x;

partialSum[t] = arr[thread_id];

if(thread_id >=size)
   return ;
for(unsigned int stride = 1; stride < blockDim.x; stride *= 2){

     if(t % (2*stride) == 0)
          partialSum[t] += partialSum[t+stride];
     __syncthreads();
}

 if(t==0)
     *result = partialSum[0];

}

int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];

    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

 float* array = nullptr;  // Pointer to the array
 float * result = nullptr;
    float value;
    int count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        float* temp = static_cast<float*>(realloc(array, (count + 1) * sizeof(float)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();

    // Print the elements in the array
    for (int i = 0; i < count; i++) {
        std::cout << array[i] << std::endl;
    }



    // host to device allocation
     float *array_device;
     float *result_device;

     result = (float*)malloc(sizeof(float));
     cudaMalloc((void**)&array_device, count * sizeof(float));
     cudaMalloc((void**)&result_device,sizeof(float));

    cudaMemcpy(array_device, array, count * sizeof(float), cudaMemcpyHostToDevice);
    dim3 threadsPerBlock(256, 1);
    dim3 numBlocks(1,1);
    ArrAdd<<< numBlocks, threadsPerBlock , threadsPerBlock.x * sizeof(float) >>>(array_device,count,result_device);

    cudaMemcpy( result, result_device, sizeof(float), cudaMemcpyDeviceToHost);

    cout<<"final sum is: "<<*result<<endl;


    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(array);

   return 0;
}


Writing Q1_2_31_2_33.cu


In [ ]:
!chmod +x Q1_2_31_2_33.cu
!nvcc Q1_2_31_2_33.cu -o Q1
!./Q1 "input.txt"

1.2
3.8
6.4
7.9
final sum is: 19.3


**Question2 (Q2)**

In [40]:

%%writefile Q2_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <fstream>
#include <vector>



using namespace std;

// Kernel definition
__global__ void BinarySearch(float *array, int size, float target, float *index, bool* found)
{
    int block_dim = 512;
    int search_interval = 16;

    if(*found == false) { // not found yet

        int thread_id = threadIdx.x;
        __shared__ float shared_array[block_dim * search_interval];


        for(int i = 0; i < search_interval; i++) {
            int shared_location = thread_id * search_interval + i;
            int location = shared_location + block_dim * search_interval * blockIdx.x;

            if(location < size) {
              shared_array[shared_location] = arr[location];
            }
            else {
              shared_array[shared_location] = INT_MAX;
            }
        }
        __syncthreads();

        int start = 0;
        int end = search_interval - 1;
        int mid = (start + end) / 2;

        float *thread_part = &shared_array[thread_id * search_interval];

        while(start <= end && *found == false)
        {
            if(thread_part[mid] < target) {
                start = mid + 1;
            }
            else if(thread_part[mid] > target) {
                end = mid - 1;
            }
            else {
                *found = true;
                *index = mid + thread_id*search_interval + block_dim * search_interval * blockIdx.x;
            }

            mid = (start + end) / 2;
        }
        if (*found == false){
          *index = -1;
        }
    }
}



int main(int argc, char *argv[])
{


    // Specify the file path
    std::string filePath = argv[1];
    float target = atof(argv[2]);
    // Open the file for reading
    std::ifstream file(filePath);

   if (!file.is_open()) {
        std::cerr << "Error opening file" << std::endl;
        return 1;
    }

 float* array = nullptr;  // Pointer to the array
 float * result = nullptr;
    float value;
    int count = 0;  // Variable to keep track of the number of elements

    // Read elements from the file
    while (file >> value) {
        // Dynamically resize the array
        float* temp = static_cast<float*>(realloc(array, (count + 1) * sizeof(float)));

        if (temp == nullptr) {
            std::cerr << "Error allocating memory" << std::endl;
            free(array);  // Free the previously allocated memory
            return 1;
        }

        array = temp;

        // Add the read value to the array
        array[count] = value;

        count++;
    }

    // Close the file
    file.close();

    // Print the elements in the array
    for (int i = 0; i < count; i++) {
        std::cout << array[i] << std::endl;
    }


      bool * flag;
    // host to device allocation
     float *array_device;
     float *result_device;
      bool * flag_device;

     result = (float*)malloc(sizeof(float));
     flag = (bool*)malloc(sizeof(bool));
     *flag = false;

     cudaMalloc((void**)&array_device, count * sizeof(float));
     cudaMalloc((void**)&result_device,sizeof(float));
     cudaMalloc((void**)&flag_device,sizeof(bool));

    cudaMemcpy(array_device, array, count * sizeof(float), cudaMemcpyHostToDevice);
        cudaMemcpy(flag_device, flag, sizeof(bool), cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(512, 1);
    dim3 numBlocks(1,1);
    BinarySearch<<< numBlocks, threadsPerBlock , threadsPerBlock.x * sizeof(float) >>>(array_device,count, target,result_device,flag_device);

    cudaMemcpy( result, result_device, sizeof(float), cudaMemcpyDeviceToHost);

    cout<<"target index : "<<*result<<endl;


    // Free the dynamically allocated memory in host
    free(array);
    free(result);

    // Free device memory
    cudaFree(array_device);
    cudaFree(array);

   return 0;
}


Overwriting Q2_2_31_2_33.cu


In [43]:
!chmod +x Q2_2_31_2_33.cu
!nvcc Q2_2_31_2_33.cu -o Q2
!./Q2 "input.txt" 1.2

1.2
3.8
6.4
7.9
target index : -1


In [ ]:
!nvprof ./Q2 "input.txt" 3.8

1.2
3.8
6.4
7.9
==10318== NVPROF is profiling process 10318, command: ./Q2 input.txt 3.8
target index : 1
==10318== Profiling application: ./Q2 input.txt 3.8
==10318== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   78.04%  13.984us         1  13.984us  13.984us  13.984us  BinarySearch(float*, int, float, float*, bool*)
                   11.61%  2.0800us         1  2.0800us  2.0800us  2.0800us  [CUDA memcpy DtoH]
                   10.36%  1.8560us         2     928ns     704ns  1.1520us  [CUDA memcpy HtoD]
      API calls:   99.64%  187.07ms         3  62.358ms  3.3460us  187.06ms  cudaMalloc
                    0.24%  443.11us         1  443.11us  443.11us  443.11us  cudaLaunchKernel
                    0.07%  132.36us       114  1.1610us     139ns  49.385us  cuDeviceGetAttribute
                    0.03%  55.657us         3  18.552us  6.7800us  27.157us  cudaMemcpy
                    0.01%  23.295us         2  